In [1]:
#Import de librerías necesarias
import pyreadr
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn import *
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import random
import time
import pandas as pd
from scipy.stats import chi2
from scipy.stats import chi2_contingency
import scipy.stats as stats
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from scipy.stats import mannwhitneyu
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from collections import defaultdict
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from scipy.stats import spearmanr, kendalltau
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model

In [2]:
#Importar los datos de R a Python
datos = pyreadr.read_r('area_desbalance_ionico.RData')

#Para comprobar que se importan correctamente los datos
#print(datos.keys())

#Este archivo tiene tres variables

#datos_anon_D: datos extraídos de los ECGS - edad, sexo, fecha, hora y 
#nombre del fichero. Hay un código (código) que identifica a cada paciente
datos_anon_DF = datos['datos_anon_DF']

#lead_anon_DF
#variables electrocardiográficas correspondiente a los ECGs de la variable anterior
leads_anon_DF = datos['leads_anon_DF']

#pacs_con_ECGs_DF
#variables de los datos del potasio, fecha de la analítica, edad del paciente, sexo
#nivel de K y código del paciente
pacs_con_ECGs_DF = datos['pacs_con_ECGs_DF']

# Normalidad - Hiperpotasemia - Hipopotasemia

In [3]:
#Primer paso: coger solo los valores de K normales y de hiperpotasemia
k_val = pacs_con_ECGs_DF.copy()

limites = [float('-inf'), 3.5, 5.2, float('inf')]
etiquetas = [1, 0, 2]

In [4]:
k_valores = k_val.copy()

k_valores['categoria'] = pd.cut(k_valores['K'], bins=limites, labels=etiquetas, right=False)

#Se reemplazan los sexos por enteros
#1: Hombre
#0: Mujer
k_valores['Sexo'] = k_valores['Sexo'].replace({'M': 1, 'F': 0})

# Eliminación de variables con datos vacíos

In [5]:
#En leads_anon_DF, se van a eliminar aquellas columnas en las que falte más del 1% de los datos

#El como máximo (el 100%) de datos que podría tener cada característica, sería igual al número total de filas que
#hay registradas
#Se calcula el 10% y se redondea sin decimales
diez_pct_datos = round(len(leads_anon_DF) * 0.01, 0)

#Ahora, se va a coger un subconjunto del dataframe en el que se eliminan las
#características cuyo número de datos sea menor al 10%

#Se cuenta el número de NaN o celdas sin datos en cada columna
celdas_sin_datos = leads_anon_DF.isnull().sum()

#Se cogen las características cuyo número de datos vacíos es mayor o igual al 10%
caracteristicas_eliminar = celdas_sin_datos[celdas_sin_datos >= diez_pct_datos].index

#Se crear el subDataFrame eliminando las columnas correspondientes
leads_anon_DF_limpio = leads_anon_DF.drop(columns = caracteristicas_eliminar)

In [6]:
#Ahora hay que asociar a cada análisis de sangre, su ECG más cercano
def ECG_mas_reciente(fecha, codigo):
     
    archivos_ECGs = datos_anon_DF.loc[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]    
    
    fechas_archivos = archivos_ECGs['date']
    horas_archivos = archivos_ECGs['time']
        
    diferencia_temporal = [abs(fecha - fecha_archivo) for fecha_archivo in fechas_archivos]
    
    ecg = (datos_anon_DF.index[(datos_anon_DF['codigo'] == codigo) & (datos_anon_DF['date'] >= (fecha - timedelta(days=5))) & (datos_anon_DF['date'] <= (fecha + timedelta(days=5)))]).tolist()
    
    if(len(diferencia_temporal) > 0):
    
        combinado = list(zip(diferencia_temporal, ecg))
        combinado = sorted(combinado, key=lambda x: x[0])

        diferencia_temporal, ecg = zip(*combinado)
        
    return ecg, diferencia_temporal

In [7]:
#Ahora, se añade la columna "Categoria" y "Sexo" a leads_anon_DF_limpio
#Se crean esas columnas en el nuevo dataframe con valores a Nan
leads_anon_DF_limpio['Sexo'] = float('NaN')
leads_anon_DF_limpio['categoria'] = float('NaN')
leads_anon_DF_limpio['K'] = float('NaN')
leads_anon_DF_limpio['Edad'] = float('NaN')
ecgs_utilizadas = {} 

for i in range(len(k_valores)):
    fecha_hora = (k_valores['Fecha'].iloc[i]).to_pydatetime()
    fecha = fecha_hora.date()
    
    indice, dist_temporal = ECG_mas_reciente(fecha, k_valores['codigo'].iloc[i])
  
    for j in range(len(indice)):
        #Si el ECGs no tiene una analítica asignada, se le pone
        if indice[j] not in ecgs_utilizadas:
            #Se actualizan los valores de las columnas "Sexo" y "Categoría"
            #de la fila correspondiente
            leads_anon_DF_limpio.at[indice[j], 'Sexo'] = k_valores['Sexo'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'Edad'] = k_valores['Edad'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
            leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]

            ecgs_utilizadas[indice[j]] = [k_valores['categoria'].iloc[i], dist_temporal[j].days]

        #Si el ECG ya tiene una analítica asociada, si la nueva analítica tiene un valor
        #de categoría diferente y se asocia con una anormalidad, se cambia
        else:
            if((dist_temporal[j].days < ecgs_utilizadas[indice[j]][1])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days
                
            if((dist_temporal[j].days == ecgs_utilizadas[indice[j]][1]) & (k_valores['categoria'].iloc[i] > ecgs_utilizadas[indice[j]][0])):
                leads_anon_DF_limpio.at[indice[j], 'categoria'] = k_valores['categoria'].iloc[i]
                leads_anon_DF_limpio.at[indice[j], 'K'] = k_valores['K'].iloc[i]
                ecgs_utilizadas[indice[j]][1] = dist_temporal[j].days                
            
#Se eliminan las filas que tengan la variable objetivo "categoria" a NaN
leads_anon_DF_limpio = leads_anon_DF_limpio.dropna(subset=['categoria'])

In [8]:
print(f'Analíticas normalidad: {(k_valores["categoria"] == 0).sum()}')
print(f'Analíticas hipopotasemia: {(k_valores["categoria"] == 1).sum()}')
print(f'Analíticas hiperpotasemia: {(leads_anon_DF_limpio["categoria"] == 2).sum()}')
print(f'Analíticas normalidad: {(leads_anon_DF_limpio["categoria"] == 0).sum()}')
print(f'Analíticas hipopotasemia: {(leads_anon_DF_limpio["categoria"] == 1).sum()}')
print(f'Analíticas hiperpotasemia: {(leads_anon_DF_limpio["categoria"] == 2).sum()}')

Analíticas normalidad: 1763
Analíticas hipopotasemia: 260
Analíticas hiperpotasemia: 750
Analíticas normalidad: 1544
Analíticas hipopotasemia: 189
Analíticas hiperpotasemia: 750


# Eliminación de características anormales

In [9]:
pparea = leads_anon_DF_limpio.filter(regex='_pparea', axis=1)
tparea = leads_anon_DF_limpio.filter(regex='_tparea', axis=1)

ppamp = leads_anon_DF_limpio.filter(regex='_ppamp', axis=1)
rpamp = leads_anon_DF_limpio.filter(regex='_rpamp', axis=1)
spamp = leads_anon_DF_limpio.filter(regex='_spamp', axis=1)
tpamp = leads_anon_DF_limpio.filter(regex='_tpamp', axis=1)

rpdur = leads_anon_DF_limpio.filter(regex='_rpdur', axis=1)
spdur = leads_anon_DF_limpio.filter(regex='_spdur', axis=1)
tpdur = leads_anon_DF_limpio.filter(regex='_tpdur', axis=1)

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=ppamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpamp.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=rpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=spdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tpdur.columns)

# Eliminación de correlaciones

In [10]:
parea = leads_anon_DF_limpio.filter(regex='_parea', axis=1)
pppparea = leads_anon_DF_limpio.filter(regex='_pppparea', axis=1)
qdur = leads_anon_DF_limpio.filter(regex='_qdur', axis=1)
sdur = leads_anon_DF_limpio.filter(regex='_sdur', axis=1)
tarea = leads_anon_DF_limpio.filter(regex='_tarea', axis=1)
tptparea = leads_anon_DF_limpio.filter(regex='(_tptparea$)', axis=1)
tptpdur = leads_anon_DF_limpio.filter(regex='(_tptpdur$)', axis=1)
st = leads_anon_DF_limpio.filter(regex='(_stend$|_st80$|_ston$)', axis=1)
stslope = leads_anon_DF_limpio.filter(regex='(_stslope)', axis=1)
stdur = leads_anon_DF_limpio.filter(regex='(_stdur$)', axis=1).drop(columns=['V2_stdur'])

leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=parea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=pppparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=qdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=sdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tarea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptparea.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=tptpdur.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=st.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stslope.columns)
leads_anon_DF_limpio = leads_anon_DF_limpio.drop(columns=stdur.columns)

# CARGA DE DATOS FINALIZADA

---

# Correlación entre variables

### Matriz de Correlación

In [11]:
matriz_correlacion = leads_anon_DF_limpio.corr(method='spearman')

#Se define un mínimo de correlación en valor absoluto del 0.7
correlacion = 0.8

#Se filtra la matrz
filtro = (matriz_correlacion.abs() >= correlacion) & (matriz_correlacion.abs() < 1)

car_colinealidad = matriz_correlacion.columns[filtro.any()]

matriz_correlacion_filtrada = matriz_correlacion.loc[car_colinealidad, car_colinealidad]

matriz_correlacion_filtrada = matriz_correlacion_filtrada.mask(matriz_correlacion_filtrada.abs() < correlacion)

matriz_correlacion_filtrada.to_excel('matriz_corr.xlsx')

### VIF

In [13]:
x_2 = leads_anon_DF_limpio.drop(['K', 'categoria'], axis = 1)
x_2 = x_2.where(pd.notna(x_2), None)

imputer = IterativeImputer(max_iter=5, sample_posterior=True, initial_strategy='mean', skip_complete=True)
x_2 = pd.DataFrame(imputer.fit_transform(x_2), columns=x_2.columns)

esc = StandardScaler()
x_2 = pd.DataFrame(esc.fit_transform(x_2), columns=x_2.columns)

VIF = pd.DataFrame()
VIF['Caracteristica'] = x_2.columns
VIF['VIF'] = [variance_inflation_factor(x_2.values, i) for i in range(len(x_2.columns))] 
VIF = VIF.sort_values('VIF', ascending=False)
VIF.to_excel('VIF.xlsx', index=True)

# Prueba Spearman

In [17]:
cars = [
    ["II_parea", "II_pamp"],
    ["II_pppparea", "II_pamp"],
    ["II_qamp", "II_qdur"],
    ["II_samp", "II_sdur"],
    ["II_qrsdur", "V5_qrsdur"],
    ["II_ston", "II_stmid"],
    ["II_ston", "II_st80"],
    ["II_ston", "II_stend"],
    ["II_stmid", "II_st80"],
    ["II_stmid", "II_stend"],
    ["II_st80", "II_stend"],
    ["II_stend", "II_stslope"],
    ["II_stdur", "V3_stdur"],
    ["II_tarea", "II_tamp"],
    ["II_tptparea", "II_tamp"],
    ["II_tdur", "II_tptpdur"],
    ["II_qtint", "V2_qtint"]
]

for i in range(len(cars)):
    spearman_corr, spearman_p_value = spearmanr(leads_anon_DF_limpio[cars[i][0]].values, 
                                            leads_anon_DF_limpio[cars[i][1]].values, nan_policy='omit')
    
    print(f'{cars[i][0]} - {cars[i][1]}: {spearman_corr}, {spearman_p_value}')

II_parea - II_pamp: 0.9486530198606639, 0.0
II_pppparea - II_pamp: 0.9302277853393132, 0.0
II_qamp - II_qdur: -0.9672045125578507, 0.0
II_samp - II_sdur: -0.9216391508002032, 0.0
II_qrsdur - V5_qrsdur: 0.7793127754922046, 0.0
II_ston - II_stmid: 0.9476840274250862, 0.0
II_ston - II_st80: 0.8833209063108172, 0.0
II_ston - II_stend: 0.8249256507917563, 0.0
II_stmid - II_st80: 0.9534938319034615, 0.0
II_stmid - II_stend: 0.9047735503148979, 0.0
II_st80 - II_stend: 0.9451193701623133, 0.0
II_stend - II_stslope: 0.6900601795793426, 0.0
II_stdur - V3_stdur: 0.9899644011348282, 0.0
II_tarea - II_tamp: 0.9717921814997652, 0.0
II_tptparea - II_tamp: 0.963616737280656, 0.0
II_tdur - II_tptpdur: 0.9080673014073167, 0.0
II_qtint - V2_qtint: 0.7202490705405327, 0.0


---